In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')



Mounted at /content/gdrive


In [ ]:
ls

gdrive/  sample_data/


In [ ]:
cd gdrive/

/content/gdrive


In [ ]:
ls


annotations/  images/  images.txt


In [ ]:
!unzip yolo.zip && rm yolo.zip

Archive:  yolo.zip
  inflating: hard_hat_workers4999.txt  
  inflating: hard_hat_workers0.txt   
  inflating: hard_hat_workers1.txt   
  inflating: hard_hat_workers2.txt   
  inflating: hard_hat_workers3.txt   
  inflating: hard_hat_workers4.txt   
  inflating: hard_hat_workers5.txt   
  inflating: hard_hat_workers6.txt   
  inflating: hard_hat_workers7.txt   
  inflating: hard_hat_workers8.txt   
  inflating: hard_hat_workers9.txt   
  inflating: hard_hat_workers10.txt  
  inflating: hard_hat_workers11.txt  
  inflating: hard_hat_workers13.txt  
  inflating: hard_hat_workers14.txt  
  inflating: hard_hat_workers15.txt  
  inflating: hard_hat_workers16.txt  
  inflating: hard_hat_workers17.txt  
  inflating: hard_hat_workers18.txt  
  inflating: hard_hat_workers19.txt  
  inflating: hard_hat_workers20.txt  
  inflating: hard_hat_workers21.txt  
  inflating: hard_hat_workers22.txt  
  inflating: hard_hat_workers23.txt  
  inflating: hard_hat_workers24.txt  
  inflating: hard_hat_workers

In [ ]:
!nvidia-smi

Fri Jul  2 03:36:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Downloading Yolo model


In [ ]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15150, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (53/53), done.
remote: Total 15150 (delta 39), reused 51 (delta 21), pack-reused 15073
Receiving objects: 100% (15150/15150), 13.50 MiB | 7.52 MiB/s, done.
Resolving deltas: 100% (10285/10285), done.
Checking out files: 100% (2036/2036), done.


In [ ]:
cd ..

/content/gdrive/My Drive/wobo/data/images


Compiling


In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/gdrive/My Drive/wobo/data/images/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:935:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1136:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
          

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

Changing params suited for our model

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivision=1/subdivision=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=2@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=35@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=35@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=35@' cfg/yolov3_training.cfg

In [ ]:
!echo -e 'Head \n Helmet' > data/obj.names
!echo -e 'classes= 2\ntrain = data/train.txt\nvalid = data/text.txt\nnames = data/obj.names\nbackup = ../' > data/obj.data 
!mkdir data/obj

Downloading initial weights

In [ ]:

!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-06-30 12:47:40--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  41.2MB/s    in 4.2s    

2021-06-30 12:47:45 (36.5 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



In [ ]:
cat data/obj.data

classes= 2
train = data/train.txt
valid = data/text.txt
names = data/obj.names
backup = ../


In [ ]:
ls

3rdparty/               darknet_images.py       net_cam_v3.sh*
backup/                 darknet.py              net_cam_v4.sh*
build/                  darknet_video.py        obj/
build.ps1               data/                   README.md
cfg/                    image_yolov3.sh*        results/
cmake/                  image_yolov4.sh*        scripts/
CMakeLists.txt          include/                src/
darknet*                json_mjpeg_streams.sh*  vcpkg.json
darknet53.conv.74       LICENSE                 video_yolov3.sh*
DarknetConfig.cmake.in  Makefile                video_yolov4.sh*


In [ ]:
import glob
images_list = glob.glob("../*.png")
print(images_list)
print(len(images_list))

['../hard_hat_workers755.png', '../hard_hat_workers715.png', '../hard_hat_workers768.png', '../hard_hat_workers763.png', '../hard_hat_workers703.png', '../hard_hat_workers71.png', '../hard_hat_workers722.png', '../hard_hat_workers732.png', '../hard_hat_workers706.png', '../hard_hat_workers728.png', '../hard_hat_workers729.png', '../hard_hat_workers708.png', '../hard_hat_workers731.png', '../hard_hat_workers736.png', '../hard_hat_workers745.png', '../hard_hat_workers762.png', '../hard_hat_workers818.png', '../hard_hat_workers773.png', '../hard_hat_workers811.png', '../hard_hat_workers782.png', '../hard_hat_workers780.png', '../hard_hat_workers798.png', '../hard_hat_workers797.png', '../hard_hat_workers808.png', '../hard_hat_workers801.png', '../hard_hat_workers829.png', '../hard_hat_workers812.png', '../hard_hat_workers785.png', '../hard_hat_workers820.png', '../hard_hat_workers835.png', '../hard_hat_workers791.png', '../hard_hat_workers778.png', '../hard_hat_workers783.png', '../hard_h

In [ ]:

file = open("data/train.txt","w")
file.write("\n".join(images_list))
file.close()


Training

In [ ]:

!./darknet detector train data/obj.data cfg/yolov3_training_test_2.cfg ../yolov3_training_test_2_last.weights -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.723638), count: 2, class_loss = 1.750816, iou_loss = 0.568269, total_loss = 2.319086 
 total_bbox = 67459, rewritten_bbox = 0.053366 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000039, iou_loss = 0.000000, total_loss = 0.000039 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.813117), count: 1, class_loss = 0.475487, iou_loss = 0.170659, total_loss = 0.646145 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.692275), count: 2, class_loss = 1.292427, iou_loss = 0.445622, total_loss = 1.738049 
 total_bbox = 67462, rewritten_bbox = 0.053363 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000310, iou_loss = 0.000000, total_loss = 0.000310 
v3 (mse

In [ ]:
cat cfg/yolov3_training_test.cfg


[net]
# Testing
batch=64
subdivisions=1
# Training
# batch=64
# subdivisions=16
width=416
height=416
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 4000
policy=steps
steps=400000,450000
scales=.1,.1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

# Downsample

[convolutional]
batch_normalize=1
filters=64
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=32
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[shortcut]
from=-3
activation=linear

# Downsample

[convolutional]
batch_normalize=1
filters=128
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[shortcut]
from=-3
activation=linear

[convolutional

In [ ]:
cat cfg/yolov3_training.cfg

[net]
# Testing
batch=64
subdivisions=1
# Training
# batch=64
# subdivisions=16
width=416
height=416
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation = 1.5
exposure = 1.5
hue=.1

learning_rate=0.001
burn_in=1000
max_batches = 4000
policy=steps
steps=400000,450000
scales=.1,.1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

# Downsample

[convolutional]
batch_normalize=1
filters=64
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=32
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[shortcut]
from=-3
activation=linear

# Downsample

[convolutional]
batch_normalize=1
filters=128
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[shortcut]
from=-3
activation=linear

[convolutional